# Анализ лояльности пользователей Яндекс Афиши

- Автор: Попов Егор Валерьевич
- Дата: 07.01.2026

### Цели и задачи проекта

**Цель**: провести исследовательский анализ данных, чтобы понять, какие пользователи с большей вероятностью возвращаются на платформу и делают заказы повторно.

**Задачи:**

1. С помощью SQL-запроса, выгрузить в датафрейм pandas необходимые данные из базы `data-analyst-afisha`, изучить общую информацию о выгруженных данных и оценить корректность выгрузки и объём полученных данных.


2. Провести предобработку данных.

    - привести выручку к единой валюте — российскому рублю;
    - изучить пропущенные значения и при необходимости обработать пропуски;
    - изучить корректность типов данных и при необходимости преобразовать;
    - изучить значения в ключевых столбцах и выявлении ошибок устранить их;
    - проверить данные на явные и неявные дубликаты;
    - изучить значения в ключевых столбцах. Для категоральных столбцов провести нормализацию, а для числовых рассмотреть их распределение и наличие в них выбросов.


3. Создать профиль пользователя по агрегированным признакам, описывающим его поведение. Для каждого пользователя найти:
    - дату первого и последнего заказа;
    - устройство, с которого был сделан первый заказ;
    - регион, в котором был сделан первый заказ;
    - билетного партнёра, к которому обращались при первом заказе;
    - жанр первого посещённого мероприятия (по полю event_type_main);
    - общее количество заказов;
    - среднюю выручку с одного заказа в рублях;
    - среднее количество билетов в заказе;
    - среднее время между заказами. 
    - добавить бинарные признаки: `is_two` — совершил ли пользователь 2 и более заказа, `is_five` — совершил ли пользователь 5 и более заказов.

   Для оценки репрезентативности сформированных профилей пользователей и проверки их на аномалые значения необходимо рассчитать:
   
    - общее число пользователей в выборке;
    - среднюю выручку с одного заказа;
    - долю пользователей, совершивших 2 и более заказа;
    - долю пользователей, совершивших 5 и более заказов;
    - статистические показатели (по общему числу заказов; по среднему числу билетов в заказе; по среднему количеству дней между покупками).
    
    
    
4. Провести исследовательский анализ данных:
    - 4.1. Исследовать признаки первого заказа и их связи с возвращением на платформу:
        - изучить распределение пользователей по различным признакам. Для этого необходимо сгрупировать пользователей по сегментам и расчитать общее количество пользователей в каждом сегменте и их долю в разрезе каждого признака;
        - проанализировать возвраты пользователей. Для каждого сегмента вычислить долю пользователей, совершивших два и более заказа;
        - проверить продуктовые гипотезы (формулировки гипотез будут приведены в разделе посвященном ИАД).
    - 4.2. Исследовать поведение пользователей через показатели выручки и состава заказа:
        - изучить количественные характеристики заказов пользователей, чтобы узнать среднюю выручку сервиса с заказа и количество билетов, которое пользователи обычно покупают;
        - проследить связь между средней выручкой сервиса с заказа и повторными заказами;
        - сравнить распределение по средней выручке с заказа в двух группах пользователей: совершившие 2–4 заказа и совершившие 5 и более заказов;
        - проанализировать влияние среднего количества билетов в заказе на вероятность повторной покупки;
    - 4.3. Исследовать временные характеристики первого заказа и их влияния на повторные покупки:
        - изучить временные параметры, связанные с первым заказом пользователей;
        - проанализировать, как день недели, в которой была совершена первая покупка, влияет на поведение пользователей;
        - изучить, как средний интервал между заказами влияет на удержание клиентов.
    - 4.4. Провести корреляционный анализ количества покупок и признаков пользователя.
    
5. Сформулировать общие выводы и рекомендации по проведённому анализу.

### Описание данных

Необходимые для анализа данные сервиса **Яндекс Афиша** представлены в базе данных PostgreSQL `data-analyst-afisha`. Подключение к БД осуществлялось с помощью программы **DBeaver**.

Схема `afisha` базы данных `data-analyst-afisha` содержит пять таблиц.

Таблица `purchases` содержит информацию о заказах билетов:

- `order_id` — идентификатор заказа;  
- `user_id` — идентификатор пользователя, совершившего заказ;  
- `created_dt_msk` — дата создания заказа по московскому времени;  
- `created_ts_msk` — дата и время создания заказа по московскому времени;  
- `event_id` — идентификатор мероприятия из таблицы `events`;  
- `cinema_circuit` — сеть кинотеатров, где проходит мероприятие. Если оно проходит не в кинотеатре, то здесь будет значение нет;  
- `age_limit` — возрастное ограничение мероприятия;  
- `currency_code` — валюта оплаты;  
- `device_type_canonical` — тип устройства, с которого был оформлен заказ (например, `mobile` — мобильные устройства, `desktop` — компьютеры);  
- `revenue` — выручка от заказа;  
- `service_name` — название билетного оператора;  
- `tickets_count` — количество купленных билетов;  
- `total` — общая сумма заказа.

Таблица `events` содержит данные о мероприятиях, доступных на платформе:

- `event_id` — идентификатор мероприятия.  
- `event_name_code` — название мероприятия в закодированном виде.  
- `event_type_description` — описание мероприятия.  
- `event_type_main` — основной тип мероприятия (театральная постановка, концерт и так далее)  
- `organizers` — организаторы мероприятия.  
- `city_id` — идентификатор города проведения мероприятия из таблицы `cities`.  
- `venue_id` — идентификатор площадки проведения мероприятия из таблицы `venues`.

Таблица `venues` содержит сведения о площадках проведения мероприятий:

- `venue_id` — идентификатор площадки;  
- `venue_name` — название площадки;  
- `address` — адрес площадки.

Таблица `city` содержит список городов, относящихся к регионам:

- `city_id` — идентификатор города;  
- `city_name` — название города;  
- `region_id` — идентификатор региона, в котором расположен город, из таблицы `regions`.

Таблица `regions` содержит список регионов, в которых проводятся мероприятия:

- `region_id` — идентификатор региона;  
- `region_name` — название региона.


### Содержимое проекта

<a id="toc"></a>
- [1. Загрузка данных и знакомство с ними](#загрузка-данных-и-знакомство-с-ними)
    - [1.1. Выгрузка в датафрейм pandas необходимых для анализа данных](#выгрузка-в-датафрейм-pandas-необходимых-для-анализа-данных)
    - [1.2. Общая информация о выгруженных данных](#общая-информация-о-выгруженных-данных)
- [2. Предобработка данных](#предобработка-данных)
    - [2.1. Приведение выручки к единой валюте](#приведение-выручки-к-единой-валюте)
    - [2.2. Проверка данных на пропущенные значения](#проверка-данных-на-пропущенные-значения)
    - [2.3 Обработка явных и неявных дубликатов](#обработка-явных-и-неявных-дубликатов)
    - [2.4. Преобразование типов данных](#преобразование-типов-данных)
    - [2.5. Изучение значений в ключевых столбцах и обработка ошибок](#изучение-значений-в-ключевых-столбцах-и-обработка-ошибок)
- [3. Создание профиля пользователя](#создание-профиля-пользователя)
    - [3.1. Построение профиля пользователя](#построение-профиля-пользователя)
    - [3.2. Оценка репрезентативности профиля пользователя](#оценка-репрезентативности-профиля-пользователя)
- [4. Исследовательский анализ данных](#исследовательский-анализ-данных)
    - [4.1. Исследование признаков первого заказа и их связи с возвращением на платформу](#исследование-признаков-первого-заказа-и-их-связи-с-возвращением-на-платформу)
        - [4.1.1. Изучение распределения пользователей по признакам](#изучение-распределения-пользователей-по-признакам)
        - [4.1.2. Анализ возвратов пользователей](#анализ-возвратов-пользователей)
        - [4.1.3. Проверка продуктовых гипотез](#проверка-продуктовых-гипотез)
    - [4.2. Исследование поведения пользователей через показатели выручки и состава заказа](#исследование-поведения-пользователей-через-показатели-выручки-и-состава-заказа)
        - [4.2.1. Исследование связи между средней выручкой сервиса с заказа и повторными заказами](#исследование-связи-между-средней-выручкой-сервиса-с-заказа-и-повторными-заказами)
        - [4.2.2. Сравнение распределения по средней выручке с заказа в выделенных группах пользователей](#cравнение-распределения-по-средней-выручке-с-заказа-в-выделенных-группах-пользователей)
        - [4.2.3. Анализ влияния среднего количества билетов в заказе на вероятность повторной покупки](#анализ-влияния-среднего-количества-билетов-в-заказе-на-вероятность-повторной-покупки)
    - [4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки](#исследование-временных-характеристик-первого-заказа-и-их-влияния-на-повторные-покупки)
        - [4.3.1. Анализ влияния дня недели, в который была совершена первая покупка на поведение пользователей](#анализ-влияния-дня-недели-в-который-была-совершена-первая-покупка-на-поведение-пользователей)
        - [4.3.2. Исследование влияния среднего интервала между заказами на удержание клиентов](#исследование-влияния-среднего-интервала-между-заказами-на-удержание-клиентов)
    - [4.4. Корреляционный анализ количества покупок и признаков пользователя](#корреляционный-анализ-количества-покупок-и-признаков-пользователя)
- [5. Общие выводы и рекомендации](#общие-выводы-и-рекомендации)
        
        

<a id="загрузка-данных-и-знакомство-с-ними"></a>
## 1. Загрузка данных и знакомство с ними

<a id="выгрузка-в-датафрейм-pandas-необходимых-для-анализа-данных"></a>
### 1.1. Выгрузка в датафрейм pandas необходимых для анализа данных

[Назад к оглавлению](#toc)

<a id="общая-информация-о-выгруженных-данных"></a>
### 1.2. Общая информация о выгруженных данных

[Назад к оглавлению](#toc)

---

### Промежуточный вывод

<a id="предобработка-данных"></a>
## 2. Предобработка данных

[Назад к оглавлению](#toc)

<a id="приведение-выручки-к-единой-валюте"></a>
### 2.1. Приведение выручки к единой валюте

[Назад к оглавлению](#toc)

<a id="проверка-данных-на-пропущенные-значения"></a>
### 2.2.  Проверка данных на пропущенные значения

[Назад к оглавлению](#toc)

<a id="обработка-явных-и-неявных-дубликатов"></a>
### 2.3. Обработка явных и неявных дубликатов

[Назад к оглавлению](#toc)

<a id="преобразование-типов-данных"></a>
### 2.4. Преобразование типов данных

[Назад к оглавлению](#toc)

<a id="изучение-значений-в-ключевых-столбцах-и-обработка-ошибок"></a>
### 2.5. Изучение значений в ключевых столбцах и обработка ошибок

[Назад к оглавлению](#toc)

<a id="создание-профиля-пользователя"></a>
## 3. Создание профиля пользователя

[Назад к оглавлению](#toc)

<a id="построение-профиля-пользователя"></a>
### 3.1 Построение профиля пользователя

[Назад к оглавлению](#toc)

<a id="оценка-репрезентативности-профиля-пользователя"></a>
### 3.2 Оценка репрезентативности профиля пользователя

[Назад к оглавлению](#toc)

<a id="исследовательский-анализ-данных"></a>
## 4. Исследовательский анализ данных

[Назад к оглавлению](#toc)

<a id="исследование-признаков-первого-заказа-и-их-связи-с-возвращением-на-платформу"></a>
### 4.1. Исследование признаков первого заказа и их связи с возвращением на платформу

[Назад к оглавлению](#toc)

<a id="изучение-распределения-пользователей-по-признакам"></a>
#### 4.1.1. Изучение распределения пользователей по признакам

[Назад к оглавлению](#toc)

<a id="анализ-возвратов-пользователей"></a>
#### 4.1.2. Анализ возвратов пользователей

[Назад к оглавлению](#toc)

<a id="проверка-продуктовых-гипотез"></a>
#### 4.1.3. Проверка продуктовых гипотез

[Назад к оглавлению](#toc)

<a id="исследование-поведения-пользователей-через-показатели-выручки-и-состава-заказа"></a>
### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа

[Назад к оглавлению](#toc)

<a id="исследование-связи-между-средней-выручкой-сервиса-с-заказа-и-повторными-заказами"></a>
#### 4.2.1. Исследование связи между средней выручкой сервиса с заказа и повторными заказами

[Назад к оглавлению](#toc)

<a id="cравнение-распределения-по-средней-выручке-с-заказа-в-выделенных-группах-пользователей"></a>
#### 4.2.2. Сравнение распределения по средней выручке с заказа в выделенных группах пользователей

[Назад к оглавлению](#toc)

<a id="анализ-влияния-среднего-количества-билетов-в-заказе-на-вероятность-повторной-покупки"></a>
#### 4.2.3. Анализ влияния среднего количества билетов в заказе на вероятность повторной покупки

[Назад к оглавлению](#toc)

<a id="исследование-временных-характеристик-первого-заказа-и-их-влияния-на-повторные-покупки"></a>
### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки

[Назад к оглавлению](#toc)

<a id="анализ-влияния-дня-недели-в-который-была-совершена-первая-покупка-на-поведение-пользователей"></a>
#### 4.3.1. Анализ влияния дня недели, в который была совершена первая покупка на поведение пользователей

[Назад к оглавлению](#toc)

<a id="исследование-влияния-среднего-интервала-между-заказами-на-удержание-клиентов"></a>
#### 4.3.2. Исследование влияния среднего интервала между заказами на удержание клиентов

[Назад к оглавлению](#toc)

<a id="корреляционный-анализ-количества-покупок-и-признаков-пользователя"></a>
### 4.4. Корреляционный анализ количества покупок и признаков пользователя

[Назад к оглавлению](#toc)

<a id="общие-выводы-и-рекомендации"></a>
## 5. Общие выводы и рекомендации

[Назад к оглавлению](#toc)